# Model assembly

## Modell Auswahl

Neuronales Netz, but WHY?????

## Modellimplementierung

###### Imports
Importiert die Bibliotheken PyTorch und Pandas

In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

###### Device
Bestimmt ob auf der Grafikkarte oder auf dem Prozessor gerechnet werden soll.
Wenn eine CUDA fähige Grafikkarte erkannt wird, wird diese als Rechengerät ausgewählt.
 Dies spart Rechenzeit, da CUDA Kerne deutlich effizienter in der Berechnung Neuronaler Netze sind.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


###### PyTorch Klassen
Definiert eine Klasse für das Datenset und eine für das Neuronale Netzwerk.

In [3]:
class LolProDataset(Dataset):
    def __init__(self, data_file):
        # Preload data into tensors
        data = pd.read_csv(data_file)
        self.labels = torch.tensor(data.pop('Win Rate').to_numpy(), device=device)
        self.features = torch.tensor(data.to_numpy(), device=device)

    def __len__(self):
        # Number of rows in the dataset
        return self.features.shape[0]

    def __getitem__(self, idx):
        # Returns item (features, label) at specific index
        x = self.features[idx]
        y = self.labels[idx]
        return (x, y)

    def split(self, test_rate):
        # Returns number of items for test and train sets by given test_rate
        testc = int(self.__len__()*test_rate)
        trainc = int(self.__len__() - testc)
        return [trainc, testc]

# Implementierung des neuronalen Netzes auf Basis der Vererbung vom nn.Modul
class LolProNetwork(nn.Module):
    def __init__(self, network_stack):
        super(LolProNetwork, self).__init__()
        self.network_stack = network_stack

    def forward(self, x):
        return self.network_stack(x)

###### Train and Test Loops
Loop in dem das Neuronale Netz trainiert und getestet wird.
Der mittlere Fehler aus jedem Batch wird genutzt um die Gewichte im Netzwerk im nächsten Batch anzupassen.
Mit Backpropagation werden die Errors durch das Netzwerk zurückgeführt und die Gewichte entsprechend angepasst werden.

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
    # Während der Trainingszeit ausgeführt
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Berechnung des Loss durch die Prediction
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test_loop(dataloader, model, loss_fn, optimizer):
    # Wird auf dem trainierten Netzwerk während der Testzeit ausgeführt
    size = len(dataloader.dataset)
    test_loss, err = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y.unsqueeze(1)).item()
            err += torch.abs(pred - y.unsqueeze(1)).sum().data
    test_loss /= size
    err /= size
    return (err.item(), test_loss)

###### Run Code
Set parameters, define Network, loss function and optimizer, load in Dataset and train the network
The Learning Rate is

Die Epochen beschreiben wie oft das training mit anderer batch Reihenfolge durchgeführt werden soll, damit keine Biase
zum Ende des Trainingssatzes entstehen. Also wie oft die Trainfunktion auf das Netz ausgeführt werden soll

Die Größe der der Batches entscheidet wie viele Zeilen für jede Neuberechnung genutzt werden sollen. Größere Batches können zu Underfitting und kleinere zu Overfitting führen
Die Learning Rate ist der Faktor der bestimmt wie weit die Weights in jedem Durchlauf an das vorherige Ergebnis angepasst werden sollen.

Der `network_stack` beschreibt den Aufbau des Netzwerkes.
Der erste Netzwerklayer wird mit den 20 Datenpunkten gefüttert.
Die Aktivierungsfunktion für den Hiddenlayer ist die ReLu Funktion
![img](https://pytorch.org/docs/stable/_images/ReLU.png)
Die Ausgabe wird mit einer Sigmoid Funktion aktiviert.

In [ ]:
lr = 0.0001 # Learning Rate
batch_size = 20 # Batch Größe (Parallel berechnete Zeilen)
epochs = 10000 # Epochen (Iterationen)

# Network
network_stack = nn.Sequential(
    nn.Linear(20, 5),
    nn.ReLU(),
    nn.Linear(5, 1),
    nn.Sigmoid()
)

# Erstelle das Model auf basis des Netzwerks und lasse es auf dem Gerät (cpu oder cuda) berechnen
model = LolProNetwork(network_stack).to(device).double()

loss_fn = nn.L1Loss() # Mean average Loss function
optimizer = torch.optim.Adam(model.parameters(), lr) # Optimizer

# Datenset laden
dataset = LolProDataset('cleanDataTop+MidS10Final.csv')
train_data, test_data = random_split(dataset, dataset.split(0.1), generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Training in jeder epoche
history = pd.DataFrame([], columns=["Epoch", "MAE", "Loss"])
for t in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer)
    res = test_loop(test_dataloader, model, loss_fn, optimizer)
    history = history.append({"Epoch": t+1, "MAE": res[0], "Loss": res[1]}, ignore_index=True)
    if (t+1)%100 == 0:
        print(f"Epoch {t+1} - MAE: {res[0]}, Loss: {res[1]}")

###### Visualisierung
Visualiserung des Trainings mit matplotlib

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history['Epoch'], history['MAE'])
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.show()

plt.plot(history['Epoch'], history['Loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
# My own stuff, gets deleted soon
my_test = [100, 5, 4, 1.5, 3.5, 9, 450, 0.5, 0.2, 600, 1.3, 0.6, 0.2, 0.3, 500, 30, 400, 0.3, 0.05, 40]
my_test_tensor = torch.tensor(my_test, device=device).double()
model(my_test_tensor)